In [7]:
import os
import glob
import re
import csv
import copy
import pandas as pd
import numpy as np
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

__version__ = "20220927"
__author__ = "L.COSTA (ATR)"


def get_inputs(debug:bool, full_dataset:bool) -> dict:
    # purpose
    # input:
    #   debug            : True (display additional information for debug) or False (basic display)
    #   full_dataset     : True/False status for using the full dataset or only a selection of variables
    # ouput:
    #   return ...       : dictionary with 'filepath' str, 'response' string as target variable for modeling, 'variables' as input variables for modeling 
    filepath = None    # full path of the input data file
    response = None    # name of the response variable (i.e. target for the model)
    useful_vars = None # list of names of all selected (useful) variables as inputs for modeling


    filepath = "C:\\Users\\to202835\\Documents\\exploitation\\formation\\db_covid19\\donnees-hospitalieres-covid-19-dep-france.csv"

    response = 'Total_Deces' # My target for the modeling

    if not full_dataset:
        # File's variables that I can use for the project
        useful_vars = ['Code_du_Departement','Date','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','Total_retour_a_domicile','Total_Deces','Code_region','Code_ISO_3166_de_la_zone','Nom_region','Nom_departement','Sexe','geo_point_2d','HospConv','SSR_USLD','autres','Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile']
        #   r2_scores_train: 0.9795083631226138
        #   r2_scores_test: 0.9788894211053454

        # Reduced set of variables considered for modeling

        useful_vars = ['Code_region', 'Code_du_Departement','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','Total_retour_a_domicile','Total_Deces','Sexe','HospConv','Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile']
        #   r2_scores_train: 0.9761588302088899
        #   r2_scores_test: 0.9755838001304158

        useful_vars = ['Code_region', 'Code_du_Departement','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','Total_retour_a_domicile','Total_Deces','Sexe','HospConv'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile'
        #   r2_scores_train: 0.9761588302088899
        #   r2_scores_test: 0.9755838001304158

        useful_vars = ['Code_region', 'Code_du_Departement','Total_retour_a_domicile','Total_Deces','Sexe','HospConv'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs'
        #   r2_scores_train: 0.9739281412849445
        #   r2_scores_test: 0.9734957171394152

        useful_vars = ['Code_region', 'Code_du_Departement','Total_retour_a_domicile','Total_Deces','Sexe'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','HospConv'
        #   r2_scores_train: 0.9739281412849445
        #   r2_scores_test: 0.9734957171394152

        useful_vars = ['Code_region', 'Code_du_Departement', 'Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs', 'Total_retour_a_domicile','Total_Deces','Sexe'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','HospConv'
        #   r2_scores_train: 0.9761588302088899
        #   r2_scores_test: 0.9755838001304158
        # We keep this settings

        useful_vars = ['Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs', 'Total_retour_a_domicile','Total_Deces','Sexe'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','HospConv', 'Code_region', 'Code_du_Departement'
        #   r2_scores_train: 0.9523048607117208
        #   r2_scores_test: 0.9511261411557373

        useful_vars = ['Code_du_Departement', 'Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs', 'Total_retour_a_domicile','Total_Deces','Sexe'] # out ,'Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','HospConv', 'Code_region'
        #   r2_scores_train: 0.9761588302089482
        #   r2_scores_test: 0.9755838001373758
        # ***** We keep this settings, optimizing quantity of variables to establish the model ****
    else:
        # **** To activate for answering question related to the most missing data ****
        useful_vars = ['Code_du_Departement','Date','Nb_actuellement_hospitalises','Nb_actuellement_en_soins_intensifs','Total_retour_a_domicile','Total_Deces','Code_region','Code_ISO_3166_de_la_zone','Nom_region','Nom_departement','Sexe','geo_point_2d','HospConv','SSR_USLD','autres','Nb_Quotidien_Admis_Hospitalisation','Nb_Quotidien_Admis_Reanimation','Nb_Quotidien_Deces','Nb_Quotidien_Retour_a_Domicile']

    del debug, full_dataset
    return {'filepath': filepath, 'response': response, 'variables': useful_vars}


def read_file_data(debug:bool, data_reading_inputs:dict) -> object:
    # purpose: Read and get data from one file
    # input:
    #   debug               : True (display additional information for debug) or False (basic display)
    #   data_reading_inputs : .... pandas dataframe path is the path of the folder that contains all raw data csv files
    # ouput:
    #   df_list             : List of all df read on csv files content, with an additional column reporting the year of the results
    df = None

    # sub-inputs variables:
    filepath = data_reading_inputs['filepath']
    variables = data_reading_inputs['variables']
    response = data_reading_inputs['response']
    avail_vars = None

    print ("> Read data")
    if filepath is None:
        print ("  - No file selected")
    else: # Read the file
        print ("  - Input file:", filepath)
        try:
            df = pd.read_csv(filepath, sep=';', encoding='latin-1', low_memory=False) # Read file with encoding latin-1 due to occurrence of non-utf8
                                                                                      # Add a low_memory=False to avoid error
        except:
            print("  CAUTION: Unable to read the file")

        if df.shape[1] == 0: # is df empty ? case of no column, whatever rows index
            df = None # we reset df to avoid further abnormal use

        if df is not None: # df has a content ...
            if variables is None: # ... but no variables to look for were defined
                print('  CAUTION: No useful variables pre-defined')
                variables = df.columns # when no useful variables are pre-defined, we call all variables available in the file
                if debug: print("variables: ", variables)
                data_reading_inputs['variables'] = variables
                avail_vars = df.columns
                print('  - ', len(variables), ' variables found in the file will be considered')
            else: # ... variables were defined to look for into the df
                # Check all useful variables are available in the raw data
                avail_vars = [] # variables pre-defined which are actually available within df
                for var in variables:
                    if var in df.columns:
                        avail_vars.append(var)

            if len(variables) != len(avail_vars):
                    print('  CAUTION: Unable to get all usefull variables') 
            else:
                # Get only useful declared variables
                df_use = df[variables] # Keep only useful vars into df
                df = df_use.copy(deep=True) # copy result into df (df is replaced only with these vars)
                del df_use

            if response not in df.columns:
                print("  CAUTION: Response variable '", response, "' is not available in df")
                df = None

    if debug and (df is not None):
        print('  - df shape  :', df.shape)
        print('  - df columns:', df.columns)
    del debug, filepath, variables, avail_vars
    return df, data_reading_inputs


def get_df_size_delta(debug, shape_1, shape_2) -> list:
    # purpose : compute and display difference of size between two df 
    # input:
    #   debug : True (display additional information for debug) or False (basic display)
    #   shape_1 : shape of first dataframe
    #   shape_2 : shape of second dataframe
    # ouput:
    #   list of [difference of number of rows: 2-1, difference of number of columns: 2-1 ]

    # sub-variables
    delta_row = shape_1[0] - shape_2[0]
    delta_col = shape_1[1] - shape_2[1]

    if debug: 
        if delta_row != 0:
            if delta_row > 0: signe = "+"
            else: signe = ""
            print('  - Change of row nb :', signe, delta_row)
        if delta_col != 0:
            if delta_col > 0: signe = "+"
            else: signe = ""
            print('  - Change of col. nb:', signe, delta_col)

    return [delta_row, delta_col]



def manage_dummy_df(debug, df: object, data_reading_inputs:dict) -> object:
    # purpose : Dummy categorical variables within the dataframe
    # input:
    #   debug : True (display additional information for debug) or False (basic display)
    #   df    : pandas dataframe with categorical variables that i'd like to dummy
    # output:
    #   df : dataframe with non categorical (copy of df) and categorical data that has the following characteristics:
    #           1. contains all columns that were not specified as categorical
    #           2. removes all the original columns in cat_df
    #           3. dummy columns for each of the categorical columns in cat_df
    #           4. if dummy_na is True - it also contains dummy columns for the NaN values
    #           5. Use a prefix of the column name with an underscore (_) for separating 

    # sub-variables
    cat_df = None
    df_col, nb_add_var, df_samp, df_samp_col, drop_var, add_var, final_df_size, init_df_size, delta = None, None, None, None, None, None, None, None, None
    response = data_reading_inputs['response']    

    if df is not None:
        init_df_size = df.shape

        print ("> Dummy cat. data")
        try:
            if debug:
                types_occur = df.dtypes.value_counts()
                print("  - types_occur:\n", types_occur)
            cat_df = df.select_dtypes(include=['object']) # Return a subset of categorical df's columns, in addition of non-categorical columns
           
            # Identify cat / non-cat columns
            list_df = df.columns
            list_df_cat = cat_df.columns
            list_df_non_cat = []
            for var in list_df:
                if var not in list_df_cat:
                    list_df_non_cat.append(var)
            if debug:
                print("  - list cat    :", list_df_cat)
                print("  - list non cat:", list_df_non_cat)
            del list_df, list_df_cat, list_df_non_cat
        except:
            print("  CAUTION: Unable to find categorical var in df") 
            cat_df = pd.DataFrame([])
        
        if debug:
            print('  - Nb of categorical data detected:', cat_df.shape[1])

        if cat_df.shape[1] == 0: # is df empty ? case of no column, whatever rows index
            print("  CAUTION: No categorical data found in raw data") 
        else:            
            for var in cat_df: # Run along the categorical data columns
                if var != response: # We exclude the response variable of this process
                    if debug:
                        print("  - var '", var, "'")
                    try:
                        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=False)], axis=1)
                        '''
                        Initially, i've limited the number of new dummy variables to create into df to 10 but it was not enough to get a meomry error at this step.
                        such finally, I reduce the file's content to only 3 years instead of a century
                        '''
                    except:
                        print("    CAUTION: Unable to concat cat. var. '", var, "'")
                        continue

        if debug and (df is not None):
            final_df_size = df.shape
            print('  - df shape      :', df.shape)
            print('  - df columns    :', df.columns)
            delta = get_df_size_delta(debug, init_df_size, final_df_size)
        
    del debug, cat_df, df_samp, df_col, nb_add_var, df_samp_col, drop_var, add_var, final_df_size, init_df_size, delta
    return df


def manage_invalid_data(debug, df: object) -> object:
    # purpose : Manage variable with infinite values; remove the column
    # input:
    #   debug : True (display additional information for debug) or False (basic display)
    #   df    : pandas dataframe with categorical variables that i'd like to dummy
    # output:
    #   df : df with infinite values replaced by nan

    # sub-variables
    a, isfinite, array = None, None, None # a is content of a column for one given variable; column with finite status for related column value; 

    if df is not None:

        print ("  > Manage invalid data")
        if debug:
            print("    - df.columns:", df.columns)

        # Manage infinite data
        if debug:
            print ("    > Manage infinite data")

        for var in df.columns:
            a = df[var]
            if a.dtypes != object:
                isfinite = np.isfinite(a)
                if not np.all(isfinite):
                    if debug:
                        print("      - ", var, "contains infinite values")
                    try:
                        if a.dtypes == 'float64':
                            df = df.drop(columns=[var]) # Drop/Remove var from df
                            if debug: print("        - float64 variable removed") # For the time being, no other solution that removed the column ; other did not work
                        else:
                            array = np.where(not(isfinite), np.nan, a) # replace infinite value by nan
                            df[var] = pd.Series(c_array)
                            if debug: print("        - Infinite values (other than float64) replaced by nan")
                    except:
                        print("      CAUTION: Unable to replace infinite values on", var, ' (', a.dtypes, ')')
                        continue

        # Manage missing numeric value; Fill numeric columns with the mean
        if debug:
            print ("    > Manage missing numerical data")

        num_vars = df.select_dtypes(include=['float', 'int']).columns
        if len(num_vars) > 0:
            for col in num_vars:
                if df[col].isnull().values.any(): # When variable contains nan to replace by mean value, then ...
                    try:
                        if debug: print("      - Add mean value on: ", col)
                        df[col].fillna((df[col].mean()), inplace=True) # ... Replace nan by mean value of the var
                    except:
                        if debug: print("      CAUTION: Unable to fill the mean for var '", col, "'") 
        else:
            if debug: print ("      - No float/int variable found in df")

    del a, isfinite, array, num_vars 
    return df


def clean_df(debug, df: object, data_reading_inputs: dict) -> object: # object is df
    # purpose : remove infinite values 
    # input:
    #   debug : True (display additional information for debug) or False (basic display)
    #   df    : pandas dataframe  dataframe
    # ouput:
    #   df : input dataframe after management of missing and infinite data
    
    # sub-variables
    df_r = None
    delta, df_size_init = None, None
    response = data_reading_inputs['response']

    print("> Clean df") # Manage missing data
    if debug:
        print("  > Impute data: Remove all empty row/column")
        print("    - response:", response)
        print("    - df.columns:", df.columns)
    try:
        if df is not None:
            df_size_init = df.shape
            df_r = df.dropna(axis=0, how='all') # Manage rows with none data ; Drop rows with all missing values
            df = df_r.dropna(axis=1, how='all') # Manage columns with none data ; Drop columns with all missing values
            delta = get_df_size_delta(debug, df_size_init, df.shape) # Result of the clean ops
    except:
        print("  CAUTION: Unable to remove empty row/column")  
    if response not in df.columns:
        print("  CAUTION: the Response variable '", response, "' is not available in the df")
        df = None

    # Drop rows with missing response values
    if debug:
        print("  > Remove rows with missing value from the response column")
        print("    - response:", response)
        print("    - df.columns:", df.columns)
    try:
        if df is not None:
            df_size_init = df.shape
            df = df.dropna(subset=[response]) # remove rows (axis=0 by default) with missing value in the column label (subset=) pred_name
            delta = get_df_size_delta(debug, df_size_init, df.shape) # Result of the clean ops
    except:
        print("  CAUTION: Unable to remove missing values from the response column")        

    # Manage invalid values
    df_size_init = df.shape
    df = manage_invalid_data(debug, df)
    delta = get_df_size_delta(debug, df_size_init, df.shape) # Result of the clean ops

    del debug, data_reading_inputs, df_r, response, delta, df_size_init
    return df


def get_X_y(debug, df: object, data_reading_inputs: dict):
    # purpose : split df into exploratory X data and response y data
    # input:
    #   debug    : True (display additional information for debug) or False (basic display)
    #   df       : pandas dataframe 
    #   response : response column's name
    # ouput:
    #   X : A matrix holding all of the variables you want to consider when predicting the response
    #   y : the corresponding response vector
    X, y = None, None

    # sub-variables
    response = data_reading_inputs['response']

    print("> Split data into X/y")
    if debug:
        print("  - response:", response)
        print("  - df.columns:", df.columns)

    # Get the Response var
    if debug: print ("  > Get y")
    if response in df.columns:
        try:
            y = df[response] # Split into explanatory and response variables (1/2) : Get response variable
            df = df.drop(columns=[response]) # Remove pred_name from df
        except:
            print("    CAUTION: Unable to get the response data in df")
            y = None
    else:
        print("    CAUTION: Unable to find the response in df")
        y = None
 
    # Get the Exploratory vars
    if debug: print ("  > Get X")
    try:
        X = df.copy(deep=True) # Split into explanatory and response variables (2/2) : Get the input variables i.e. at this level just a copy of df
    except:
        print("    CAUTION: Unable to get the exploratory variables (X)")
        X = None

    if debug and (y is not None):
        print("    - y shape:", y.shape)
    if debug and (X is not None):
        print("    - X shape:", X.shape)

    del debug, df, data_reading_inputs, response
    return X, y


def get_model(debug, X: object, y: object, test_ratio=.3):
    # function: Give a prediction model according to its X/y_test/train inputs/outputs values
    # input:
    #   debug      : True (display additional information for debug) or False (basic display)
    #   X          : explanatory variables object
    #   y          : response variable object
    #   test_ratio : proportion of the dataset to include in the test split, between 0.0 and 1.0; default value = 0.3
    # output:
    #   model : linear regression model object from sklearn
    #   score : Merge of mean square error value between Train and Test data set according to the proposed model
    #   list of X_train and y_train
    #   list of X_test and y_test
    model, score = None, 0
    X_train, X_test, y_train, y_test = None, None, None, None

    # sub-variables
    y_pred, acc_score_train, acc_score_test, r2_scores_train, r2_scores_test = None, None, None, None, None

    if (X is not None) and (y is not None):

        print("> Get model")

        # Split into train and test X/y data set to establish the model and score it
        print("  > Split Train / Test")
        try:
            if debug: print("    - X/y size:", X.shape, y.shape)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42) 
            if debug: print("    - Train X/y size:", X_train.shape, y_train.shape)
            if debug: print("    - Test  X/y size:", X_test.shape, y_test.shape)
            split = True
        except:
            print("  CAUTION: Unable to split X/y into train and test data set")
            split = False

        # Work on dtype
        print("  > Type of data")

        if debug: print("    - y:", y.dtypes)
        
        recensed_type = {}
        for var in X.columns:
            tip = X[var].dtypes
            if tip not in recensed_type:
                recensed_type[tip] = 1
            else:
                recensed_type[tip] = recensed_type[tip] +1
        if len(recensed_type) > 0:
            for key, value in recensed_type.items():
                if debug: print("    - X", key, " : x", value)
            if debug: print("    - Over", X.shape[1], " columns")


        # Establish model
        print("  > Modeling")      
        #try:
        if split:
            if debug: print("    - Train X/y size:", X_train.shape, y_train.shape)
            # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict / https://www.codegrepper.com/code-examples/python/reg.predict+python
            model = LinearRegression().fit(X_train, y_train) # Fit linear model ; further methods: https://scikit-learn.org/stable/modules/linear_model.html
            # It does not work

            # https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification
            # and https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
            #clf = linear_model.Ridge(alpha=1.0)
            #Ridge(alpha=1.0)
            #model = clf.fit(X_train, y_train)
            # It does not work too

            # Get the model's score
            mdl_score = model.score(X_train, y_train) # Return the coefficient of determination of the prediction.
            if debug: print("    - Train mdl_score:", mdl_score)
            del mdl_score
            if debug: print("    - Test  X/y size:", X_test.shape, y_test.shape)
            y_pred = model.predict(X_test)
        else:
            y_pred = None
        del split

        # Evaluate this model
        if y_pred is not None:
            if debug: print ("    - Model found")

        # Get metrics with a model by Regression
        print("  > Metrics") 
        if y_pred is not None:
            # Accuracy_score (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
            # Confusion matrix (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) not appropriate for my purpose
            # Common pitfalls (https://scikit-learn.org/stable/common_pitfalls.html): mean_sqaured_error and r2_score

            '''
            # According to https://stackoverflow.com/questions/37367405/python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
            # ... Accuracy score is only for classification problems
            acc_score_train = accuracy_score(y_train, model.predict(X_train))
            acc_score_test = accuracy_score(y_test, model.predict(X_test))
            if debug: print("    - acc_score_train:", acc_score_train)
            if debug: print("    - acc_score_test:", acc_score_test)
            '''
            # always according to https://stackoverflow.com/questions/37367405/python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
            # ... For regression problems you can use: R2 Score, MSE (Mean Squared Error), RMSE (Root Mean Squared Error).
            r2_scores_train = r2_score(y_train, model.predict(X_train))
            r2_scores_test = r2_score(y_test, model.predict(X_test))
            print("    - r2_scores_train:", r2_scores_train)
            print("    - r2_scores_test:", r2_scores_test)

            score = r2_scores_train

    del debug, X, y, test_ratio, y_pred, acc_score_train, acc_score_test, r2_scores_train, r2_scores_test
    return model, score, [X_train, y_train], [X_test, y_test]


def coef_weights(debug, model, X_train) -> object: # object is a df:
    # function: get model's coefficients
    # input:
    #   debug      : True (display additional information for debug) or False (basic display)
    #   model      : model for which we are looking coefficients
    #   X_train    : the training data
    # output:
    #   coefs_df   : dataframe with model's coefficients; that can be used to understand the most influential coefficients
    #                in a linear model by providing the coefficient estimates along with the name of the variable attached to the coefficient.

    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = model.coef_ # get coefficients of the linear model 
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)

    del debug, model, X_train
    return coefs_df


def df_logical_no(debug, df, column):
    # function: add a logical no values of a given column in a df
    # input:
    #   debug      : True (display additional information for debug) or False (basic display)
    #   df         : initial dataframe to modify
    #   column     : name of the df's column to compute a logical no for
    # output:
    #   df         : df completed with the column of logical no of the given column
    #   New_column_name : name of the new column

    liste, r, no_b = None, None, None
    if debug: print('   - LOGICAL NO:', column) 
    try:
        liste = df[column].values.tolist()
        for i, j in enumerate(liste):
            if j is None:
                liste[i] = -1
        complement = np.add(liste, 1) 
        no_liste = np.where(complement > 1, False, np.where(complement == 0, None, True))

        New_column_name = column + '_logical_no'
        df[New_column_name] = pd.Series(no_liste)
        if debug: print('     - df: -> no', column, '\n')
    except:
        print("    - Unable to compute logical no of column '" + column + "'")

    del debug, column, liste, r, no_b
    return df, New_column_name


def divide_serieA_by_serieB(debug, df, col_names, new_col, min_qty_col_B) -> object: # object is a df:
    # function: divide values of two columns from of df and add result into this df
    # input:
    #   debug      : True (display additional information for debug) or False (basic display)
    #   df         : initial dataframe to modify
    #   cols       : names of df's column for numerator of the division and denominator of the division
    #   new_col    : name of new df's column added with result of the division
    # min_qty_col_B: Minimum value to consider on denominator do perform the computation
    # output:
    #   df         : df completed with the column of logical no of the given column
 
    division = []    
    listeA = df[col_names[0]].values.tolist()
    listeB = df[col_names[1]].values.tolist()
    
    for i, A in enumerate (listeA):
        try:
            if listeB[i] >= min_qty_col_B:
                r = 100* listeA[i]/listeB[i] # given in percentage
            else:
                r = None
            if (i<10) or (r >99.9):
                if debug: print('i=',i, '-', listeA[i], '/', listeB[i], '=', r)
        except:
            r = None
            continue
        division.append(r)

    df[new_col] = pd.Series(division) # Add the result of the operation
    df = df.sort_values(new_col, ascending=False) # sorted df by the result

    del debug, col_names, new_col, listeA, listeB, division, min_qty_col_B
    return df


def find_value_at_target_quantil(debug, target_quantil: float, df_column_data: object, nb_valid_data=None):
    # function: Find the quantil value closest to the target quantil of a df column dataset
    # input:
    #   debug           : True (display additional information for debug) or False (basic display)
    #   target_quantil  : target quantil (%)
    #   df_column_data  : dataset of a df's column
    #   nb_valid_data   : (option) ; size of the valid data to consider in the column ; by default the full size of df_column_data is considered
    # output:
    #   found_value     : value of the df_column_data that reach the target_quantil of data at or below this value
    #   found_quantil   : actual quantil value (normally close to the target_quantil) that reach the closest the target quantil
    
    found_value, found_quantil = 0, 0 # Primary variables declaration
    delta, min_delta, moving_value, moving_value_float, moving_quantil = 100, 100, 0, 0, 0 # sub-variables declaration

    if nb_valid_data is None:
        nb_valid_data = len(df_column_data)

    if debug: print('>>> target_quantil =', target_quantil) # for instance 99.7% 'target quantil'
    for moving_value in range(0, 10000, 1): # we run the 'moving value' from 0 to 100% by 0.01%-step
        moving_value_float = moving_value/100
        moving_quantil = 100 * df_column_data[df_column_data <= moving_value_float].count() / nb_valid_data #  get 'moving quantil' of 'value' below the 'moving value'
        delta = abs(target_quantil - moving_quantil) # we compare the 'moving quantil' to the 'target quantil' we are trying to reach
        if debug: print('    for moving_value_float:', "{:.3f}".format(moving_value_float), '\t we reach moving_quantil:', "{:.3f}".format(moving_quantil), ' => delta ', "{:.3f}".format(delta), 'vs min_delta:', "{:.3f}".format(min_delta))
        if delta <= min_delta: # when the gap is minimum, it means we find value of the 'value' for which we reach the targeted 'target quantil'
            min_delta = delta
            found_value = moving_value_float
            found_quantil = moving_quantil
            if debug: print('    change of delta')
    if debug: print('    value:', found_value)

    del debug, target_quantil, df_column_data, delta, min_delta, moving_value, moving_value_float, moving_quantil
    return found_quantil, found_value


def get_selected_rate(debug, df, inputs, min_sample_size):
    # function: on a selected variable of the dataframe, provide stats (max, average, median) and find value at three target quantil
    # input:
    #   debug         : True (display additional information for debug) or False (basic display)
    #     df          : dataframe to cope with
    #  inputs         : dictionnary with result's titles as keys and a list with df's column names for numerator and denominator as values = {title1: [col_N1, col_D1], title2: [...] ...}
    # min_sample_size : Minimum value to consider on denominator do perform the computation
    # output:
    #   outputs       : dictionary with result's titles as keys and a sub-dictionary with target percentage and result values

    outputs = {}
    tgt_pct = [99.7, 95.4, 68.3] # According to Gaussian theory: %-values related to 3sigma, 2sigma, 1sigma
    found_ratio_of_result, found_xfr_rate_float = None, None 

    for title in inputs:
        print("  > Provide stats related to", title, ":")
        outputs[title] = title
        num_denum = inputs[title] # get column's title of [numerator; denominator]
        result_name = entree.replace(" ", "_")
        df = divide_serieA_by_serieB(debug, df, num_denum, result_name, min_sample_size)
        results = {}
        column = df[result_name]
        nb_valid_data = len(df) - column.isna().sum()
        ratio_valid_result = 100 * column.isna().sum() / len(df) # Compute ratio of computed result 'result_name' over the full size of data
        print('    - Computed values on ', "{:.2f}".format(ratio_valid_result), '%', 'of the full dataset')
        maximum_value = column.max() # Compute max value of 'result_name'
        median_value = column.median() # Compute median value of 'result_name'        
        average_value = column.mean() # Compute average value of 'result_name'
        print('    - maximum:', "{:.2f}".format(maximum_value), '\n    - average:', "{:.2f}".format(average_value), '\n    - median :', "{:.2f}".format(median_value))
        for target_ratio_of_result in tgt_pct: # we are looking the value (%) of xfr for which we have thr (serie here-above) below this value (%).
                                                 # for instance, we look for the threshold value of xfr rate for which 95.4% of the xfr rate values are below this threshold value
            found_ratio_of_result, found_xfr_rate_float = find_value_at_target_quantil(debug, target_ratio_of_result, column, nb_valid_data)
            if (found_xfr_rate_float != 0) and (abs(target_ratio_of_result - found_ratio_of_result)<= 0.1): # accuracy to find the result = 0.1%
                #tgt, value = str(target_ratio_of_result), str(found_xfr_rate_float)
                results[target_ratio_of_result] = found_xfr_rate_float # compile results in a dictionary
                print('    - ', "{:.2f}".format(target_ratio_of_result), '%', 'of the transfer rate values are equal or below', "{:.2f}".format(found_xfr_rate_float), '%') # Display results
        outputs[title] = results

    del debug, df, inputs, min_sample_size, tgt_pct, found_ratio_of_result, found_xfr_rate_float, num_denum, result_name, nb_valid_data, ratio_valid_result, maximum_value, median_value, average_value
    return outputs


if __name__ == "__main__":

    print (10*"-", "PROJECT #01 / START", 10*"-", "\n")
 
    # Display rules
    # no tab / no symbol: High level title
    # '>' Action step, tab length according to sub-level
    # '-' Result, tab lentgth according to the sub-level
    # 'CAUTION': abnormal/unexpected behaviour/result

    # High level variables of the script for dev
    debug = False  # if True, display additional information along the run of the app
    use_full_data_set = False # read and customize analysis according to the use of the whole dataset or only a selection to optimize modeling effort
    coef_df, most_missing_cols, df_context, stats_result = None, None, None, None
    
    if debug:
        print('INFO! Debug mode is active!')
    if use_full_data_set:
        print('INFO! Use full dataset and realize appropriate analysis')
    else:
        print('INFO! Use selected dataset and realize appropriate analysis')

    # Select input data
    data_reading_inputs = get_inputs(debug, use_full_data_set) # get information on inputs to read and work on

    # Gather data
    df, data_reading_inputs = read_file_data(debug, data_reading_inputs) # : read data file and get pandas df with only requested raw data
    df_init_copy = df.copy(deep=True)

    # Manage dummy data
    df = manage_dummy_df(debug, df, data_reading_inputs) # replace cat column by related dummy columns

    # Clean data
    df = clean_df(debug, df, data_reading_inputs) # drop rows / columns with all missing values + replace infinite values by nan

    # Split into Response y / Exploratory variables X
    X, y = get_X_y(debug, df, data_reading_inputs)

    # modeling
    model, score, Xy_train, Xy_test = get_model(debug, X, y)


    # Questions
    print('> Processing for providing stats')

    if use_full_data_set:
        # Is there any parameters more missing than others ?
        print("  > Define most missing data:")
        print("    Over", len(df_init_copy), "of data entries reported")
        print("    Find here-below the ratio of missing data by categories of data:")
        df_init_copy_num_missing_rate = 100 *  df_init_copy.isna().sum() / len(df_init_copy) # "df_init_copy.isna().sum()" can be replaced by "df_init_copy.isnull().sum()""
        # we can have the same result with "df_init_copy_num_missing_mean = df_init_copy.isna().mean().round(4) * 100"
        print(df_init_copy_num_missing_rate.sort_values(ascending=False))
        # Conversely, we could have the opposite result, i.e. available data ratio with "df_init_copy_num_rate = 100*df_init_copy.count() / len(df_init_copy)""
    
    else:
        # Which variables have the most effect on the model ?
        print('  > Define impact of variables on the model:')
        coef_df = coef_weights(debug, model, Xy_train[0])
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        print('    ', coef_df)


        inputs = {'rate of patients in critical car over hospitalization': ['Nb_actuellement_en_soins_intensifs', 'Nb_actuellement_hospitalises'],\
                  'rate of female that died due to covid-19': ['Sexe_Femme', 'Total_Deces'],\
                  'rate of male that died due to covid-19':['Sexe_Homme', 'Total_Deces']}
        min_sample_size = 10
        stats_result = get_selected_rate(debug, df, inputs, min_sample_size)

        '''
        print("  > Provide stats related to rate of female that died due to covid-19:")
        result_name = 'ratios_mortalite_femme'
        nb_min_actuellement_hospitalises = 10
        df = divide_serieA_by_serieB(debug, df, 'Sexe_Femme', 'Total_Deces', result_name, nb_min_actuellement_hospitalises)

        column = df[result_name]
        nb_valid_data = len(df) - column.isna().sum()
        ratio_valid_result = 100 * column.isna().sum() / len(df) # Compute ratio of computed result 'result_name' over the full size of data
        print('    - Computed values on ', "{:.2f}".format(ratio_valid_result), '%', 'of the full dataset')
        maximum_value = column.max() # Compute max value of 'result_name'
        median_value = column.median() # Compute median value of 'result_name'        
        average_value = column.mean() # Compute average value of 'result_name'
        print('    - maximum:', "{:.2f}".format(maximum_value), '\n    - average:', "{:.2f}".format(average_value), '\n    - median :', "{:.2f}".format(median_value))
        serie_thr = [99.7, 95.4, 68.3] # TBD ...Define thresholds to compute a ratio of input above these thresholds
        for target_ratio_of_result in serie_thr: # we are looking the value (%) of xfr for which we have thr (serie here-above) below this value (%).
                                                 # for instance, we look for the threshold value of xfr rate for which 40% of the xfr rate values are below this threshold value
            found_ratio_of_result, found_xfr_rate_float = find_value_at_target_quantil(debug, target_ratio_of_result, column, nb_valid_data)
            if (found_xfr_rate_float != 0) and (abs(target_ratio_of_result - found_ratio_of_result)<= 0.1):
                print('    - ', "{:.2f}".format(target_ratio_of_result), '%', 'of the transfer rate values are equal or below', "{:.2f}".format(found_xfr_rate_float), '%') # Display results
 
        print("  > Provide stats related to rate of male that died due to covid-19:")
        result_name = 'ratios_mortalite_homme'
        nb_min_actuellement_hospitalises = 10
        df = divide_serieA_by_serieB(debug, df, 'Sexe_Homme', 'Total_Deces', result_name, nb_min_actuellement_hospitalises)

        column = df[result_name]
        nb_valid_data = len(df) - column.isna().sum()
        ratio_valid_result = 100 * column.isna().sum() / len(df) # Compute ratio of computed result 'result_name' over the full size of data
        print('    - Computed values on ', "{:.2f}".format(ratio_valid_result), '%', 'of the full dataset')
        maximum_value = column.max() # Compute max value of 'result_name'
        median_value = column.median() # Compute median value of 'result_name'        
        average_value = column.mean() # Compute average value of 'result_name'
        print('    - maximum:', "{:.2f}".format(maximum_value), '\n    - average:', "{:.2f}".format(average_value), '\n    - median :', "{:.2f}".format(median_value))
        serie_thr = [99.7, 95.4, 68.3] # TBD ...Define thresholds to compute a ratio of input above these thresholds
        for target_ratio_of_result in serie_thr: # we are looking the value (%) of xfr for which we have thr (serie here-above) below this value (%).
                                                 # for instance, we look for the threshold value of xfr rate for which 40% of the xfr rate values are below this threshold value
            found_ratio_of_result, found_xfr_rate_float = find_value_at_target_quantil(debug, target_ratio_of_result, column, nb_valid_data)
            if (found_xfr_rate_float != 0) and (abs(target_ratio_of_result - found_ratio_of_result)<= 0.1):
                print('    - ', "{:.2f}".format(target_ratio_of_result), '%', 'of the transfer rate values are equal or below', "{:.2f}".format(found_xfr_rate_float), '%') # Display results
        '''

    del debug, use_full_data_set, data_reading_inputs, df, df_init_copy, coef_df, most_missing_cols, df_context, stats_result
    print ('\n' + 10*"-", "PROJECT #01 /  END ", 10*"-", "\n")



---------- PROJECT #01 / START ---------- 

INFO! Use selected dataset and realize appropriate analysis
> Read data
  - Input file: C:\Users\to202835\Documents\exploitation\formation\db_covid19\donnees-hospitalieres-covid-19-dep-france.csv
> Dummy cat. data
> Clean df
  > Manage invalid data
> Split data into X/y
> Get model
  > Split Train / Test
  > Type of data
  > Modeling
  > Metrics
    - r2_scores_train: 0.9761588302089482
    - r2_scores_test: 0.9755838001373758
> Processing for providing stats
  > Define impact of variables on the model:
                                     est_int         coefs     abs_coefs
107                           Sexe_Tous  4.629496e+08  4.629496e+08
106                          Sexe_Homme  4.629496e+08  4.629496e+08
105                          Sexe_Femme  4.629495e+08  4.629495e+08
60               Code_du_Departement_57  2.284012e+08  2.284012e+08
78               Code_du_Departement_75  2.284010e+08  2.284010e+08
97               Code_du_Departeme

TypeError: 'str' object does not support item assignment